In [1]:
from rdkit import Chem
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import Draw
IPythonConsole.ipython_useSVG=False

import os
import json
import rapidjson

from typing import NoReturn

In [2]:
def create_dir_for_images(folder_name: str) -> NoReturn:
    if not os.path.exists(folder_name):
        os.mkdir(folder_name)

def create_filename(num: int) -> str:
    return str(num + 1) + '.png'

In [3]:
INPUT_FILENAME = 'PubChem.json'
FOLDER_NAME = 'images'
OUTPUT_FILENAME = 'num_of_images.json'

# read json from file
with open(INPUT_FILENAME, 'r') as json_file:
    data = rapidjson.load(json_file, 
                          parse_mode=rapidjson.PM_TRAILING_COMMAS)

# create folder for images
create_dir_for_images(FOLDER_NAME)

# draw images and save information about number and SMILES in json
json_output = []
for num, mol_datum in enumerate(data):
    mol = Chem.MolFromSmiles(mol_datum['isosmiles'])
    if mol:
        canvas = Draw.rdMolDraw2D.MolDraw2DCairo(*(512, 512))
        canvas.drawOptions().setAtomPalette({-1:(0,0,0)})
        Draw.rdMolDraw2D.PrepareAndDrawMolecule(canvas, mol)
        canvas.WriteDrawingText(os.path.join(FOLDER_NAME, create_filename(num)))
        mol_json = {
            'smiles': mol_datum['isosmiles'], 
            'image_num': num + 1
        }
        json_output.append(mol_json)
        
# save json to file
with open(OUTPUT_FILENAME, 'w') as outfile:
    json.dump(json_output, outfile, indent='\t')

RDKit WARNING: [23:08:53] WARNING: not removing hydrogen atom without neighbors
[23:08:53] WARNING: not removing hydrogen atom without neighbors
RDKit ERROR: [23:09:23] Explicit valence for atom # 1 Cl, 7, is greater than permitted
[23:09:23] Explicit valence for atom # 1 Cl, 7, is greater than permitted
RDKit ERROR: [23:09:24] Explicit valence for atom # 1 Br, 3, is greater than permitted
[23:09:24] Explicit valence for atom # 1 Br, 3, is greater than permitted
RDKit ERROR: [23:09:24] Explicit valence for atom # 1 Br, 5, is greater than permitted
[23:09:24] Explicit valence for atom # 1 Br, 5, is greater than permitted
RDKit ERROR: [23:09:25] Explicit valence for atom # 1 Cl, 3, is greater than permitted
[23:09:25] Explicit valence for atom # 1 Cl, 3, is greater than permitted
RDKit WARNING: [23:09:25] WARNING: not removing hydrogen atom without neighbors
[23:09:25] WARNING: not removing hydrogen atom without neighbors
RDKit ERROR: [23:09:27] Explicit valence for atom # 1 Si, 8, is gr